<a href="https://colab.research.google.com/github/siddharthbharthulwar/CV-COVID/blob/master/lungsegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x 
import tensorflow as tf

print(tf.version.VERSION)

TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.2


In [3]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 1058 (delta 91), reused 124 (delta 25), pack-reused 857
Receiving objects: 100% (1058/1058), 188.60 MiB | 35.52 MiB/s, done.
Resolving deltas: 100% (476/476), done.
Checking out files: 100% (269/269), done.


In [0]:
class Dataset:

    def __init__(self):
        
        self.df = pd.read_csv('/content/covid-chestxray-dataset/metadata.csv')
        self.patients = dict.fromkeys(list(set(self.df['finding'])), [])
        for i in range(2, self.df['patientid'].max()): #for some reason, dataset starts at patientid 2
            patient = Patient(i, self.df)
            self.patients[patient.finding].append(patient)

In [0]:
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

class Patient:

    def __init__(self, patientID, df):
        
        self.patientID = patientID
        self.fileNames = []
        self.offsets = []
        self.dates = []
        self.images = []

        for index, row in df.iterrows(): 

            if row['patientid'] == self.patientID:

                self.fileNames.append(row['filename'])
                self.offsets.append(row['offset'])
                self.age = row['age'] 
                self.finding = row['finding'] #assuming a patient has the same finding for all scans
                self.dates.append(row['date']) #assuming chronological order of entries per patient
                

        for filename in self.fileNames:

            img = cv.imread('/content/covid-chestxray-dataset/images/' + filename)
            self.images.append(img)

    def show(self):

      for i in range(0, len(self.images)):

          plt.imshow(self.images[i])
          plt.title("Patient " + str(self.patientID) + " on " + self.dates[i] + " | Offset = " + str(self.offsets[i]))
          plt.show()


In [7]:
!git clone https://github.com/imlab-uiip/lung-segmentation-2d


Cloning into 'lung-segmentation-2d'...
remote: Enumerating objects: 51, done.
remote: Total 51 (delta 0), reused 0 (delta 0), pack-reused 51
Unpacking objects: 100% (51/51), done.


In [12]:
import numpy as np


dataset = Dataset()
a = dataset.patients['COVID-19'][1].images
images = np.empty((len(a), 1546, 1800, 3))

for i in range(0, len(a)):

    images[i] = cv.resize((256, 256))




TypeError: ignored

In [9]:
model = tf.keras.models.load_model('/content/lung-segmentation-2d/trained_model.hdf5')
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 